In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.ensemble import GradientBoostingClassifier
import rce
from scipy.stats import sem
import Datasets as DS

In [2]:
dataset_name = 'diabetes'
num_instances = 20
time_limit = 1000
rho = 0.05
unc_type = 'linf'

# clf_dict = {'linear': [0], 'cart': [3, 5, 10], 'rf': [5, 10, 20, 50, 100],
#            'gbm': [5, 10, 20, 50, 100], 'mlp': [(10,), (10, 10, 10), (50,), (100,)]}

clf_dict = {'linear':[0], 'cart':[3,5,10], 'rf':[5,10,20]}

## Load dataset

In [3]:
data = getattr(DS,dataset_name)
df = data('../data/')

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
X = df.iloc[:,:-1]
y = df['Outcome']
df.head()

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1.0


## Robust counterfcatual explanation

In [ ]:
fnamefull = './results_%s/%s_results_%s.txt' % (dataset_name,dataset_name,unc_type)

num_iterations_dict = {(i, j):[] for i in clf_dict.keys() for j in clf_dict[i]}
comp_time_dict = {(i, j):[] for i in clf_dict.keys() for j in clf_dict[i]}
dist_early_stops = {(i, j):[] for i in clf_dict.keys() for j in clf_dict[i]}
early_stops_iter = {(i, j):[] for i in clf_dict.keys() for j in clf_dict[i]}


for clf_type in clf_dict.keys():
    for param in clf_dict[clf_type]:

        if clf_type == 'cart':
            clf = DecisionTreeClassifier(max_depth=param).fit(X, y)
        elif clf_type == 'rf':
            clf = RandomForestClassifier(max_depth=3, random_state=0, n_estimators=param).fit(X, y)
        elif clf_type == 'mlp':
            clf = MLPClassifier(hidden_layer_sizes=param, activation='relu', random_state=0, max_iter=10000).fit(X, y)
        elif clf_type == 'gbm':
            clf = GradientBoostingClassifier(n_estimators=param, learning_rate=1.0, max_depth=2, random_state=0).fit(X, y)
        elif clf_type == 'linear':
            clf = LogisticRegression(random_state=param).fit(X, y)

        for i in range(20):
            print(f'######## Iteration number: {i} ########')
            np.random.seed(i)
            u = pd.DataFrame([X.iloc[i, :]])
            
            
            if clf_type == 'linear':
                it = False
            else:
                it = True
            
            final_model, num_iterations, comp_time, x_, solutions_master_dict = rce.generate(clf, X, y, '../experiments/results_%s' % dataset_name, clf_type, 'binary', u, list(u.columns), [], [], {}, [], [], [], rho,
                             unc_type=unc_type, iterative=it, time_limit=time_limit)
            if comp_time>time_limit:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!!!!!\n", i)

            solution_subopt, dist = rce.find_maxrad(x_, clf_type, 'results_%s' % dataset_name, x_.columns, [], [], {}, [], [], [], clf.predict(u)[0], unc_type)
            if dist + rho/100 < rho:
                best_dist = 0
                for i in range(len(solutions_master_dict)):
                    x_ = solutions_master_dict[i]['sol']
                    solution_subopt_i, dist_i = rce.find_maxrad(x_, clf_type, 'results_%s' % dataset_name, x_.columns, [], [], {}, [], [], [], clf.predict(u)[0], unc_type)
                    if dist_i >= best_dist:
                        best_dist = dist_i
                print(best_dist)
                dist_early_stops[(clf_type, param)].append(best_dist)
                early_stops_iter[(clf_type, param)].append(num_iterations)
                print('\n\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ ERROR @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n\n')

            else:
                num_iterations_dict[(clf_type, param)].append(num_iterations)
                comp_time_dict[(clf_type, param)].append(comp_time)

            # write results to .txt file
            num_iterations_list = num_iterations_dict[(clf_type, param)] 
            comp_time_list = comp_time_dict[(clf_type, param)] 
            dist_early_stops_list = dist_early_stops[(clf_type,param)]
            early_stops_iter_list = early_stops_iter[(clf_type, param)]

            txt = '{0}: \t {1} \t {2:.2f} ({3:.2f}) \t {4:.2f} ({5:.2f}) \t {6} \t {7:.3f} ({8:.3f}) \t {9:.2f} ({10:.2f}) '.format(clf_type,
                                      param,
                                      np.mean(comp_time_list),
                                        sem(comp_time_list),
                                        np.mean(num_iterations_list), 
                                        sem(num_iterations_list), 
                                        len(dist_early_stops_list),
                                        np.mean(dist_early_stops_list),
                                        sem(dist_early_stops_list), 
                                        np.mean(early_stops_iter_list),
                                        sem(early_stops_iter_list))
        with open(fnamefull, 'a') as f:   
            print(txt,file=f)

######## Iteration number: 0 ########
linear tables saved.
embedding validity constraints...
validity constraint embedded.
######## Iteration number: 1 ########
linear tables saved.
embedding validity constraints...
validity constraint embedded.


/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',


######## Iteration number: 2 ########
linear tables saved.
embedding validity constraints...
validity constraint embedded.


/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


######## Iteration number: 3 ########
linear tables saved.
embedding validity constraints...
validity constraint embedded.


...

######## Iteration number: 4 ########
rf tables saved.

### Starting the RANDOM FOREST iterative approach ###
time limit: 1000


------------------------ Iteration: 0 ------------------------
Optimizing the master problem...
solution master [0.0, 0.6884422110552764, 0.3278688524590164, 0.3535353535353536, 0.19858156028368795, 0.70567320419309, 0.9436379163111042, 0.20000000000000007] generated in  0.4 s
--> Distance to the factual instance: 0.06334831596656232
--> Distance to the border: 1e-05
Optimizing the adversarial problem...
Set parameter PoolSearchMode to value 1
Status: optimal
solution adv problem [-0.0500000000001819, 0.7314424206170997, 0.2778688524588233, 0.40353535353563075, 0.2075427819647757, 0.6556732041929081, 0.9936379163112861, 0.21339384956741014] generated in  0.1 s
Status adversarial problem: True


------------------------ Iteration: 1 ------------------------
Optimizing the master problem...
solution master [0.0, 0.7763919098472343, 0.3278688524590164, 0.3484848

--> Distance to the border: 0.04758
Optimizing the adversarial problem...
Set parameter PoolSearchMode to value 1
Status: optimal
solution adv problem [-0.0500000000001819, 0.8914722954181116, 0.2778688524588233, 0.346066676323062, 0.15100972792981793, 0.5923248882262442, 0.9936379163112861, 0.1499999999997499] generated in  0.0 s
Status adversarial problem: True


------------------------ Iteration: 12 ------------------------
Optimizing the master problem...
solution master [0.0, 0.8414722954179297, 0.32786885245901654, 0.3535353535353536, 0.2015466468129703, 0.6423248882265277, 0.9436379163111042, 0.20000000000000007] generated in  1.1 s
--> Distance to the factual instance: 0.1559951708919357
--> Distance to the border: 0.04906
Optimizing the adversarial problem...
Set parameter PoolSearchMode to value 1
Status: optimal
solution adv problem [-0.0500000000001819, 0.8914722954181116, 0.2778688524588233, 0.3475492195876768, 0.15154664681278973, 0.5923248882262442, 0.9936379163112861, 

/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/tabearober/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


######## Iteration number: 5 ########
rf tables saved.

### Starting the RANDOM FOREST iterative approach ###
time limit: 1000


------------------------ Iteration: 0 ------------------------
Optimizing the master problem...
solution master [0.3823629481887565, 0.5829145728640697, 0.6065573770491803, 0.0, 0.0, 0.4463587416505561, 0.05251921434648921, 0.1583433387970672] generated in  0.2 s
--> Distance to the factual instance: 0.16142726235251892
--> Distance to the border: 1e-05
Optimizing the adversarial problem...
Set parameter PoolSearchMode to value 1
Status: optimal
solution adv problem [0.43236218972005847, 0.5655778765676587, 0.5841273948550654, -0.007061777258282893, -0.03699763603526662, 0.49635950011961794, 0.04607097655539041, 0.2083425803283692] generated in  0.0 s
Status adversarial problem: True


------------------------ Iteration: 1 ------------------------
Optimizing the master problem...
solution master [0.29411764705882354, 0.7485024792343893, 0.6065573770491803, 0.

--> Distance to the border: 0.03804
Optimizing the adversarial problem...
Set parameter PoolSearchMode to value 1
Status: optimal
solution adv problem [0.2441176470586015, 0.8763919098475981, 0.5565573770489891, -0.018185927312629246, 0.02497499776472978, 0.5463587416509199, 0.04253227378819702, 0.1463707049972527] generated in  0.0 s
Status adversarial problem: True


------------------------ Iteration: 12 ------------------------
Optimizing the master problem...
solution master [0.29411764705882354, 0.8037838463930326, 0.6065573770491803, -3.410605131648481e-13, -0.0024069387808361986, 0.496358741650738, 0.05251921434648921, 0.20002000447027513] generated in  4.6 s
--> Distance to the factual instance: 0.38813483920551717
--> Distance to the border: 0.0274
Optimizing the adversarial problem...
Set parameter PoolSearchMode to value 1
Status: optimal
solution adv problem [0.2581201195372387, 0.753783846392821, 0.5565573770489891, -0.030055768822307982, 0.035610427419328516, 0.546358741